# Dados ausentes

Muitas vezes as fontes de dados estão incompletas, o que significa que você terá dados ausentes, você tem 3 opções básicas para preencher os dados ausentes (você terá que tomar pessoalmente a decisão sobre qual é a abordagem certa:

* Basta manter os pontos de dados ausentes.
* Elimine os pontos de dados ausentes (incluindo a linha inteira)
* Preencha-os com algum outro valor.

Vamos cobrir exemplos de cada um desses métodos!

## Mantendo os dados ausentes
Alguns algoritmos de aprendizado de máquina podem lidar facilmente com dados ausentes, vamos ver como é:

In [1]:
from pyspark.sql import SparkSession
# Pode demorar um pouco em um computador local
spark = SparkSession.builder.appName("missingdata").getOrCreate()

In [2]:
df = spark.read.csv("ContainsNull.csv",header=True,inferSchema=True)

In [3]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



Observe como os dados permanecem nulos.

## Elimine os dados ausentes

Você pode usar as funções .na para dados ausentes. O comando drop possui os seguintes parâmetros:

     df.na.drop (how = 'any', thresh = None, subset = None)
    
     * param how: 'any' ou 'all'.
    
         Se for 'qualquer', elimine uma linha se ela contiver nulos.
         Se 'all', elimina uma linha apenas se todos os seus valores forem nulos.
    
     * param thresh: int, default None
    
         Se especificado, elimina as linhas que têm menos de valores não nulos `thresh`.
         Isso substitui o parâmetro `how`.
        
     * subconjunto de parâmetros:
         lista opcional de nomes de colunas a serem considerados.

In [4]:
# Elimine qualquer linha que contenha dados ausentes
df.na.drop().show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [5]:
# Deve ter pelo menos 2 valores NÃO nulos
df.na.drop(thresh=2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [6]:
df.na.drop(subset=["Sales"]).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [7]:
df.na.drop(how='any').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [8]:
df.na.drop(how='all').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



## Preencha os valores ausentes

Também podemos preencher os valores ausentes com novos valores. Se você tiver vários nulos em vários tipos de dados, o Spark é realmente inteligente o suficiente para combinar os tipos de dados. Por exemplo:

In [9]:
df.na.fill('NEW VALUE').show()

+----+---------+-----+
|  Id|     Name|Sales|
+----+---------+-----+
|emp1|     John| null|
|emp2|NEW VALUE| null|
|emp3|NEW VALUE|345.0|
|emp4|    Cindy|456.0|
+----+---------+-----+



In [10]:
df.na.fill(0).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| null|  0.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



Normalmente você deve especificar quais colunas deseja preencher com o parâmetro de subconjunto

In [11]:
df.na.fill('No Name',subset=['Name']).show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John| null|
|emp2|No Name| null|
|emp3|No Name|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+



Uma prática muito comum é preencher os valores com o valor médio da coluna, por exemplo:

In [12]:
from pyspark.sql.functions import mean
mean_val = df.select(mean(df['Sales'])).collect()

# Formatação aninhada estranha do objeto Row!
mean_val[0][0]

400.5

In [13]:
mean_sales = mean_val[0][0]

In [14]:
df.na.fill(mean_sales,["Sales"]).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [19]:
df.na.fill(df.select(mean(df['Sales'])).collect()[0][0],['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

